Train : 1024x1024 , Test : 224x224
최대공약수 32

import

In [34]:
import pandas as pd
import numpy as np
import cv2
import os

Dataframe & rle decoding function

In [53]:
df=pd.read_csv('./train.csv')

# RLE 디코딩 함수
def rle_decode(mask_rle, shape):
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

Split train_img to 4X4 (size : 224x224)

In [90]:
train_directory=os.listdir('./train_img')
img_path = df.iloc[:, 1]

for file in train_directory:
    empty_matrix=[[[[0 for _ in range(224)] for _ in range(224)] for _ in range(3)] for _ in range(16)]
    image=cv2.imread('./train_img/'+file)
    count=0
    
    filename=file.rstrip('.png')
    for i in range(4):
        for j in range(4):
            empty_matrix[count]=image[224*i:224*i+223,224*j:224*j+223,0:3]
            cv2.imwrite('./preprocessed_train_img/{path}{by}{count}.jpg'.format(path=filename,by='_',count=count),empty_matrix[count])
            count=count+1

Split mask img to 4X4 (size 224)

In [89]:
mask_list=df[[df.columns[0],df.columns[2]]].to_numpy()
for mask in mask_list:
    mask_name=mask[0]
    mask_image=rle_decode(mask[1],(1024,1024))
    count=0
    empty_matrix=[[[[0 for _ in range(224)] for _ in range(224)] for _ in range(3)] for _ in range(16)]
    for i in range(4):
        for j in range(4):
            empty_matrix[count]=mask_image[224*i:224*i+223,224*j:224*j+223]
            cv2.imwrite('./decode_mask_img/{path}{by}{count}.jpg'.format(path=mask_name,by='_',count=count),mask_image*255)
            count=count+1